In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline

In [ ]:
def grid_neighbors(i,j):
    return(set([str(i)+","+str(j-1),
            str(i)+","+str(j+1),
            str(i+1)+","+str(j),
            str(i-1)+","+str(j)]))

In [ ]:
def Qpaths(i,j,k):
    if(k==1):
        return(set([str(i-3)+","+str(j),
                str(i-2)+","+str(j),
                str(i-2)+","+str(j+1),
                str(i-1)+","+str(j),
                str(i-1)+","+str(j+1)]))
    if(k==2):
        return(set([str(i-2)+","+str(j),
                str(i-1)+","+str(j),
                str(i-1)+","+str(j+1),
                str(i)+","+str(j-1)]))
    if(k==3):
        return(set([
            str(i)+","+str(j-3),
            str(i)+","+str(j-2),
            str(i)+","+str(j-1),
            str(i+1)+","+str(j-2),
            str(i+1)+","+str(j-1)]))
    if(k==4):
        return(set([str(i)+","+str(j-2),
                str(i)+","+str(j-1),
                str(i+1)+","+str(j-1),
                str(i-1)+","+str(j)]))

In [ ]:
def dependency_graph(F,verbose=False):
    particles = []
    N,M = F.shape

    for i in range(N):
        for j in range(M):
            if(F[i,j]==1):
                particles.append(str(i)+","+str(j))

    floor = [str(N-1)+","+str(i) for i in range(M)]
    rest = list(set(particles).difference(set(floor)))
    
    G = nx.DiGraph()
    G.add_nodes_from(particles)
    N,M = F.shape
    for node1p in G.nodes:
        node1 =  [int(i) for i in node1p.split(",")]
        for node2p in G.nodes:
            node2 =  [int(i) for i in node2p.split(",")]
            if(node1[0]>node2[0] and node1[1]>node2[1]):
                G.add_edge(node1p,node2p)
            if(node1[0]==N-1 and node2[0]!=N-1):
                G.add_edge(node1p,node2p)

    flag = 0
    iteration = 0
    while(flag==0):
        iteration += 1
        if(verbose):
            print("")
            print("Iteration",iteration)
        flag = 1


        Gnew = G.copy()

        #Rule 1
        for node1 in G.subgraph(rest):
            i = int(node1.split(",")[0])
            j = int(node1.split(",")[1])
            vecPred = grid_neighbors(i,j).intersection(set(G.predecessors(node1)))

            if(len(vecPred)==0):
                vecPos = grid_neighbors(i,j).intersection(set(G.nodes))
                vecPosIter = vecPos.copy()
                for node2 in vecPosIter:
                    if(node2 in G.successors(node1)):
                        for node3 in G.successors(node2):
                            vecPos.discard(node3)
                        vecPos.discard(node2)
                vecPosIter = vecPos.copy()
                if(len(vecPos)==0):
                    if(verbose):
                        print("There are no possible neighbors for", node1, "so we add a self-loop according to Rule 1.")
                    Gnew.add_edge(node1,node1)
                for node2 in vecPosIter:
                    vecRem = {node2}.union(G.successors(node2))
                    if(vecPos.issubset(vecRem)):
                        if(verbose):
                            print("Adding edge from", node2, "to", node1, "with Rule 1")
                        Gnew.add_edge(node2,node1)
                        flag = 0

        #Rule 2
        for node1 in G.subgraph(rest):
            i = int(node1.split(",")[0])
            j = int(node1.split(",")[1])
            aPaths = {1,2,3,4}
            for k in range(1,5):
                if(len(set(G.predecessors(node1)).intersection(Qpaths(i,j,k)))!=0):
                    aPaths.discard(k)
            if(len(aPaths)==0):
                if(verbose):
                    print("Adding edge from", node1, "to", node1, "with Rule 1")
                Gnew.add_edge(node1,node1)
            else:
                if(verbose):
                    print("The availables paths of", node1,"are",aPaths)
                caminos = Qpaths(i,j,aPaths.pop())
                while(len(aPaths)!=0):
                    caminos = caminos.intersection(Qpaths(i,j,aPaths.pop()))
                if(verbose):
                    print("and their intersection is",caminos)
            for node2 in caminos:
                if(node2 in G.nodes):
                    if(verbose):
                        print("Adding edge from", node1, "to", node2, "with Rule 1")
                    Gnew.add_edge(node1,node2)



        G = Gnew.copy()
    return(G)

In [ ]:
def random_pattern(N,M,p=0.5):
    A = np.random.binomial(1, p, N*M).reshape(N,M)
    Z = np.zeros((N,3))
    A = np.append(A,Z,axis=1)
    A = np.append(Z,A,axis=1)
    Z = np.zeros((3,M+6))
    A = np.append(Z,A,axis=0)
    Z = np.ones((1,M+6))
    A = np.append(A,Z,axis=0)
    A = np.array(A,dtype=int)
    return(A)

In [ ]:
def draw_graph(G,F,drawFloor=False):
    particles = []
    N,M = F.shape

    for i in range(N):
        for j in range(M):
            if(F[i,j]==1):
                particles.append(str(i)+","+str(j))

    floor = [str(N-1)+","+str(i) for i in range(M)]
    rest = list(set(particles).difference(set(floor)))
    
    temporal = [(y,[int(y.split(",")[1]),N-int(y.split(",")[0])]) for y in particles]
    positions = dict(temporal)
    if(drawFloor==True):
        nx.draw(G,pos = positions, with_labels=True)
    else:
        nx.draw(G.subgraph(rest),pos = positions, with_labels=True)
    plt.show()

In [ ]:
def draw_pattern(F):
    Frand = F.copy()
    N,M = F.shape
    for i in range(N):
        for j in range(M):
            if(F[i,j]==1):
                Frand[i,j] = 30*(0.9+np.random.random())

    fig =  plt.figure(figsize=(5,5))
    plt.imshow(1-Frand, cmap='gray')
    plt.axis(False)
    plt.show()

In [ ]:
def Realization2DLA(H,verbose=False,drawGraph=False,drawPattern=False):
    F = H.copy()
    N,M = F.shape
    particles = []
    for i in range(N):
        for j in range(M):
            if(F[i,j]==1):
                particles.append(str(i)+","+str(j))

    floor = [str(N-1)+","+str(i) for i in range(M)]
    rest = list(set(particles).difference(set(floor)))

    sequence = []
    paths = []
    G = dependency_graph(F)
    while(len(rest)!=0):
        Gold = G.copy()
        G = dependency_graph(F)
        if(drawGraph):
            draw_graph(G,F)
            print(set(G.edges).difference(set(Gold.edges)))
            print(nx.is_directed_acyclic_graph(G))
            print(list(nx.simple_cycles(G)))
        if(drawPattern):
            draw_pattern(F)
            
        removable_nodes = []
        
        paths_removable = []

        for node1 in G.subgraph(rest):
            if(len(list(G.successors(node1)))==0):
                i = int(node1.split(",")[0])
                j = int(node1.split(",")[1])
                aPaths = {1,2,3,4}
                for k in range(1,5):
                    if(len((set(rest).union(floor)).intersection(Qpaths(i,j,k)))!=0):
                        aPaths.discard(k)
                if(len(aPaths)!=0):
                        removable_nodes.append(node1)
                        paths_removable.append(aPaths.pop())
                                                    
       
        for node in removable_nodes:
            Ftemp = F.copy()
            i = int(node.split(",")[0])
            j = int(node.split(",")[1])
            Ftemp[i,j]=0
            Gtemp = dependency_graph(Ftemp)
            if(not nx.is_directed_acyclic_graph(Gtemp)):
                if(verbose):
                    print("Removing", node, "because its removal creates cycles")
                removable_nodes.remove(node)
        
        if(verbose):
            print("Removable nodes:", removable_nodes)
        
        if(len(removable_nodes)==0):
            print("Non realizable pattern")
            return(0,0)
        
        node = removable_nodes[0]
        i = int(node.split(",")[0])
        j = int(node.split(",")[1])
        F[i,j]=0
        rest.remove(node)
        sequence.append(node)
        paths.append(paths_removable[0])
        #print(node)

    sequence.reverse()
    paths.reverse()
    print("Realizable pattern")
    return(sequence,paths)

In [ ]:
def particle_adjacency_graph(F,verbose=False, drawGraph = False):
    particles = []
    N,M = F.shape
    
    for i in range(N):
        for j in range(M):
            if(F[i,j]==1):
                particles.append(str(i)+","+str(j))
            
    G = nx.Graph()        
    G.add_nodes_from(particles)

    for node1 in G.nodes:
        for node2 in G.nodes:
            i = int(node2.split(",")[0])
            j = int(node2.split(",")[1])
            if node1 in grid_neighbors(i,j):
                if(verbose == True):
                    print("Agregando arista entre",node1, "y", node2)
                G.add_edge(node1,node2)
    if(verbose == True):
        print(G.edges())
    
    if(drawGraph==True):
        temporal = [(y,[int(y.split(",")[1]),N-int(y.split(",")[0])]) for y in particles]
        positions = dict(temporal)
        nx.draw(G,pos = positions, with_labels=True)
    
    return(G)

In [ ]:
def update():
    if(step[0] < len(sequence)):
        Node = [int(i)-2 for i in sequence[step[0]].split(",")]
        NodeFin = [int(i) for i in sequence[step[0]].split(",")]
        if(Pos[0] < Node[0]):
            F[Pos[0],Pos[1]] = 0
            Pos[0] += 1
            F[Pos[0],Pos[1]] = 2
        elif(Pos[1]<Node[1]):
            F[Pos[0],Pos[1]] = 0
            Pos[1] += 1
            F[Pos[0],Pos[1]] = 2      
        else: 
            if(Pos == NodeFin):
                F[Pos[0],Pos[1]] = 1
                Pos[0] = 0
                Pos[1] = 0
                F[Pos[0],Pos[1]] = 2
                step[0] += 1
            else:
                if(paths[step[0]]==1):
                    if(Pos==Node):
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 3
                    elif(Pos[0]==Node[0] and Pos[1]==Node[1]+1):
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 3
                    elif(Pos[0]==Node[0] and Pos[1]==NodeFin[1]):
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 3
                    else:
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 3
                if(paths[step[0]]==2):
                    if(Pos==Node):
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 4
                    elif(Pos[0]==Node[0] and Pos[1]==Node[1]+1):
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 4
                    elif(Pos[0]==Node[0]+1 and Pos[1]==Node[1]+1):
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 4
                    else:
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 4
                if(paths[step[0]]==3):
                    if(Pos==Node):
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 5
                    elif(Pos[0]==Node[0]+1 and Pos[1]==Node[1]):
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 5
                    elif(Pos[0]==NodeFin[0] and Pos[1]==Node[1]):
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 5
                    else:
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 5
                if(paths[step[0]]==4):
                    if(Pos==Node):
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 6
                    elif(Pos[0]==Node[0]+1 and Pos[1]==Node[1]):
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 6
                    elif(Pos[0]==Node[0]+1 and Pos[1]==Node[1]+1):
                        F[Pos[0],Pos[1]] = 0
                        Pos[0] += 1
                        F[Pos[0],Pos[1]] = 6
                    else:
                        F[Pos[0],Pos[1]] = 0
                        Pos[1] += 1
                        F[Pos[0],Pos[1]] = 6

In [ ]:
def animate(i):
    update()
    DLA_plot.set_data(palette[F])
    return DLA_plot

**Important** The algorithms work only with patterns $F$ that have a margin of 3 unoccupied cells arround the figue.

* Function `dependency_graph()` is used to build the dependency graph of a given pattern.
* Function `draw_pattern()` produces a figure of the given pattern
* Function `draw_graph()` produces a figure with the dependency graph of a given pattern
* Function `Realization2DLA()` produces a sequence that realizes a given figure under the 2-DLA dynamics. 

### Example 1


In [ ]:
J = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,1,1,1,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,1,1,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,0,1,0,0,0],
    [0,0,0,1,1,1,1,1,1,1,1,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
])
draw_pattern(J)

In [ ]:
F = J.copy()
G = dependency_graph(F)
draw_graph(G,F)

The following cell verifies that the graph is acyclic.

In [ ]:
print(nx.is_directed_acyclic_graph(G))
list(nx.simple_cycles(G))

In [ ]:
Realization2DLA(F)

The following block creates an animation of the realization of the figure under the 2-DLA dynamics

In [ ]:
F = J.copy()
numPart = sum(sum(F))
sequence,paths = Realization2DLA(F)
step = [0]
Pos = [0,0]
F[Pos[0],Pos[1]] = 2
palette = np.array([[255, 255, 255],   # black
                    [  0,   0,   0],   # white
                    [255,   0,   0],   # red
                    [255, 255,   0],   # yellow
                    [  0, 255,   0],   # green
                    [  0, 255, 255],   # cyan
                    [  0,   0, 255]])  # red

N,M = F.shape
Z = np.ones((1,M))
F = np.append(np.zeros((N-1,M)),Z,axis=0)
F = np.array(F,dtype=int)
fig =  plt.figure(figsize=(5,5))
DLA_plot = plt.imshow(palette[F])

anim = animation.FuncAnimation(fig,animate,frames=numPart*N,interval=50)
HTML(anim.to_jshtml())

## Example 2

The second example is a non-realizable pattern.

In [ ]:
J = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,1,1,1,0,0,0],
    [0,0,0,0,0,0,1,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0,0,1,0,0,0],
    [0,0,0,1,1,1,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1],
])
draw_pattern(J)

In [ ]:
F = J.copy()
G = dependency_graph(F)
draw_graph(G,F)

It is not realizable as the dependency graph contains cycles.

In [ ]:
print(nx.is_directed_acyclic_graph(G))
list(nx.simple_cycles(G))

In [ ]:
Realization2DLA(F)

## Example 3

The third example is a non realizable pattern.

In [ ]:
J = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,1,1,1,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,1,1,0,1,0,0,0,0],
    [0,0,0,1,1,1,1,1,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,1,0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,1,0,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
])
draw_pattern(J)

In [ ]:
F = J.copy()
G = dependency_graph(F)
draw_graph(G,F)

In this case the graph is not acyclic, as it contains a self-loop in node 7,7

In [ ]:
print(nx.is_directed_acyclic_graph(G))
list(nx.simple_cycles(G))

In [ ]:
Realization2DLA(F)

## Example 4

Our code also provides a function to create randomized patterns.

In [ ]:
J = random_pattern(15,9,p=0.75)
draw_pattern(J)

In [ ]:
F = J.copy()
Realization2DLA(F)